In [ ]:
import os
from database.milvus_cloud_db.zilliz_retriever import semantic_search, initialize_retriever
from database.milvus_cloud_db.zilliz_retriever_b import initialize_retriever_b, semantic_search_b
from services.crud  import create_mental_health_db# database
from services.calculator_node import MentalHealthCalculator


from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.agents import Tool
from langchain.tools import StructuredTool

from llm_model.llm import llm_model

from typing import TypedDict, Optional, List, Dict, Any, Union
from pydantic import BaseModel, Field, model_validator
import random
import json
from pyppeteer import launch
import asyncio
from tavily import TavilyClient
from bson import ObjectId 

In [ ]:
# Initialize retriever
init_result = initialize_retriever(
    zilliz_uri="https://in03-d9c913fef2464e7.serverless.aws-eu-central-1.cloud.zilliz.com",
    zilliz_token="db_d9c913fef2464e7:Sh9&pO+<)v{>{0]1",
    collection_name= "mental_health_emori",
    embedding_model= "all-MiniLM-L6-v2",
    use_cuda= False
)
print(init_result)

In [ ]:
zilliz_uri = os.getenv("ZILLIZ_URI_B")
zilliz_token = os.getenv("ZILLIZ_TOKEN_B")

initialize_retriever_b(
    zilliz_uri=zilliz_uri,
    zilliz_token=zilliz_token,
    collection_name="sentiment_collection_emori",
    embedding_model="all-MiniLM-L6-v2",
    use_cuda=False
)

## STATE GRAPH

In [ ]:
class MainState(TypedDict):
    user_query: str
    answer: Optional[str]
    user_id: str
    past_output: Optional[List[Dict[str, str]]] 
    
    # PATH A 
    semantic_search_a_results: Optional[List[Dict[str, str]]]
    graded_documents: Optional[List[Dict[str, Union[str, int]]]] 
    label: Optional[str]

    # PATH B 
    semantic_search_b_results: Optional[List[Dict[str, Union[str, float]]]] 
    intensity_score: Optional[Dict[str, Any]]  #
    top_k_results: Optional[List[Dict[str, Union[str, float]]]]  
    
    # for calculator
    user_scores: Optional[Dict[str, float]]  
    user_decay_scores: Optional[Dict[str, float]]  
    last_update_timestamp: Optional[str]  
    calc_result: Optional[float]
    
    warning_text: Optional[str] 
    evaluation_result: Optional[str] # we use rule based evaluation
    evaluation_feedback: Optional[str]
    

## SCHEMA LLM ANSWER

### NODE